In [1]:
#Initialising the NLTK package
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
#Importing fake news detection packages
import numpy as np
import pandas as pd
import re
import string
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
stemmer = nltk.SnowballStemmer("english")
stopword=set(stopwords.words('english'))
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,  classification_report, mean_squared_error,  mean_absolute_error, confusion_matrix, r2_score

In [3]:
#Loading the dataset
news_dataset = pd.read_csv(r"/content/Fake_News_Dataset.csv",encoding='latin1',error_bad_lines=False)

<ipython-input-3-1b20ea574abd>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  news_dataset = pd.read_csv(r"/content/Fake_News_Dataset.csv",encoding='latin1',error_bad_lines=False)


In [4]:
#Information of the dataset
news_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2201 entries, 0 to 2200
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      2201 non-null   int64 
 1   title   2181 non-null   object
 2   text    2201 non-null   object
 3   label   2201 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 68.9+ KB


In [5]:
news_dataset.tail()

,id,title,text,label
2196,2435,Exclusive: Koch brothers will not use funds to...,"NEW YORK (Reuters) - The Koch brothers, the mo...",0
2197,2437,Rahm Emanuel Invents New ID to Ensure Illegals...,Chicago Mayor Rahm Emanuel has moved forward w...,0
2198,2438,"Veto, filibuster threats ahead of vote next we...",The GOP-led Senate is expected to vote next we...,0
2199,2440,Nancy Reagan remembered at funeral for fierce ...,"SIMI VALLEY, Calif. (Reuters) - First lady Mic...",0
2200,2444,Republican defends border-adjustment tax after...,WASHINGTON (Reuters) - A top Republican in Con...,0


In [7]:
#Data pre-processing step
def data_preprcss(text):
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('@', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('<.*?>+', '', text)
    text = str(text).lower()
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text

In [8]:
#Applying Data pre-processing function to clean the data
news_dataset["text"] = news_dataset["text"].apply(data_preprcss)

In [9]:
#Assigning feature and target variables
ftr = news_dataset["text"]
tar = news_dataset["label"]

In [10]:
#Divide the dataset into training and test data
X_train, X_test, Y_train, Y_test = train_test_split(ftr,tar, test_size=0.20)

In [11]:
#Converting the text data into vectorized form
vctr = TfidfVectorizer()
XVect_train = vctr.fit_transform(X_train)
XVect_test = vctr.transform(X_test)

In [12]:
#Replace NA for target data by 0
Y_train=Y_train.fillna(0)
Y_test=Y_test.fillna(0)

In [14]:
#Decision Tree classifier
import warnings
with warnings.catch_warnings(record=True):
    modl1 =DecisionTreeClassifier()
    modl1.fit(XVect_train, Y_train)
    Y_prdc1 = modl1.predict(XVect_test)
    print('Classification Report for Decision Tree classifier-> \n', classification_report(Y_test, Y_prdc1))
    print('R-squared score for Decision Tree classifier->',r2_score(Y_test, Y_prdc1))
    print('Mean Absolute Error for Decision Tree classifier->', mean_absolute_error(Y_test, Y_prdc1))
    print('Root Mean Squared Error for Decision Tree classifier->', np.sqrt(mean_squared_error(Y_test, Y_prdc1)))

Classification Report for Decision Tree classifier-> 
               precision    recall  f1-score   support

           0       0.85      0.90      0.88       235
           1       0.88      0.82      0.85       206

    accuracy                           0.86       441
   macro avg       0.87      0.86      0.86       441
weighted avg       0.86      0.86      0.86       441

R-squared score for Decision Tree classifier-> 0.4534187151414998
Mean Absolute Error for Decision Tree classifier-> 0.1360544217687075
Root Mean Squared Error for Decision Tree classifier-> 0.36885555678165877


In [15]:
#Random Forest classifier
import warnings
with warnings.catch_warnings(record=True):
    modl2 =RandomForestClassifier()
    modl2.fit(XVect_train, Y_train)
    Y_prdc2 = modl2.predict(XVect_test)
    print('Classification Report for Random Forest classifier-> \n', classification_report(Y_test, Y_prdc2))
    print('R-squared score for Random Forest classifier->',r2_score(Y_test, Y_prdc2))
    print('Mean Absolute Error for Random Forest classifier->', mean_absolute_error(Y_test, Y_prdc2))
    print('Root Mean Squared Error for Random Forest classifier->', np.sqrt(mean_squared_error(Y_test, Y_prdc2)))

Classification Report for Random Forest classifier-> 
               precision    recall  f1-score   support

           0       0.83      0.92      0.88       235
           1       0.90      0.79      0.84       206

    accuracy                           0.86       441
   macro avg       0.87      0.86      0.86       441
weighted avg       0.87      0.86      0.86       441

R-squared score for Random Forest classifier-> 0.4443090270605248
Mean Absolute Error for Random Forest classifier-> 0.1383219954648526
Root Mean Squared Error for Random Forest classifier-> 0.3719166512336502


In [16]:
#Logistic Regression classifier
import warnings
with warnings.catch_warnings(record=True):
    modl3 =LogisticRegression()
    modl3.fit(XVect_train, Y_train)
    Y_prdc3 = modl3.predict(XVect_test)
    print('Classification Report for Logistic Regression classifier-> \n', classification_report(Y_test, Y_prdc3))
    print('R-squared score for Logistic Regression classifier->',r2_score(Y_test, Y_prdc3))
    print('Mean Absolute Error for Logistic Regression classifier->', mean_absolute_error(Y_test, Y_prdc3))
    print('Root Mean Squared Error for Logistic Regression classifier->', np.sqrt(mean_squared_error(Y_test, Y_prdc3)))

Classification Report for Logistic Regression classifier-> 
               precision    recall  f1-score   support

           0       0.87      0.89      0.88       235
           1       0.87      0.84      0.86       206

    accuracy                           0.87       441
   macro avg       0.87      0.86      0.87       441
weighted avg       0.87      0.87      0.87       441

R-squared score for Logistic Regression classifier-> 0.4625284032224748
Mean Absolute Error for Logistic Regression classifier-> 0.13378684807256236
Root Mean Squared Error for Logistic Regression classifier-> 0.3657688451366004


In [17]:
#Gradient Boosting classifier
import warnings
with warnings.catch_warnings(record=True):
    modl4 =GradientBoostingClassifier()
    modl4.fit(XVect_train, Y_train)
    Y_prdc4 = modl4.predict(XVect_test)
    print('Classification Report for Gradient Boosting classifier-> \n', classification_report(Y_test, Y_prdc4))
    print('R-squared score for Gradient Boosting classifier->',r2_score(Y_test, Y_prdc4))
    print('Mean Absolute Error for Gradient Boosting classifier->', mean_absolute_error(Y_test, Y_prdc4))
    print('Root Mean Squared Error for Gradient Boosting classifier->', np.sqrt(mean_squared_error(Y_test, Y_prdc4)))

Classification Report for Gradient Boosting classifier-> 
               precision    recall  f1-score   support

           0       0.94      0.92      0.93       235
           1       0.91      0.93      0.92       206

    accuracy                           0.93       441
   macro avg       0.92      0.93      0.92       441
weighted avg       0.93      0.93      0.93       441

R-squared score for Gradient Boosting classifier-> 0.6993802933278249
Mean Absolute Error for Gradient Boosting classifier-> 0.07482993197278912
Root Mean Squared Error for Gradient Boosting classifier-> 0.2735506022160966
